# Calculo de principales indicadores usando python

A continuación se presentan los coódigos para el cálculo de los principales indicadores de la ENIGH 2020. Estan escritos para que el usuario los ejecute sin necesidad de cambio.

Para realizar este ejercicio solo seran necesatios los módulos de **pandas** y **numpy**

## Set-up

In [1]:
# Importamos librerias que vamos a utilizar
import pandas as pd
import os

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Creamos directorios
data = '/home/jovyan/work/02_indicadores_desigualdad/00_enigh/00_data/'
outputs = '/home/jovyan/work/02_indicadores_desigualdad/00_enigh/03_outputs/'

### Ejecutamos script con funciones necesarias

In [3]:
%run 'functions.py' 

## Lectura de tablas de datos

Para leer las tablas que necesitamos para el analisis, se creo una funcíon, **enigh**, que nos ayuda a traernos direcatmente de la pagina de la enigh la tabla del año que necesitemos, sin necesidad de guardar el arhivo en nuestro espacio de trabajo.

En este ejercicio, tomaremos la tabla *concentrado hogar* del año *2020* y lo guardamos en un dataframe

In [4]:
df = enigh('concentradohogar','2020')

___

## 2. Cálculo de indicadores de ingreso
### 2.1. Ingreso corriente total promedio trimestral por hogar en deciiles de hogares y su coeficiente de GINI

A continuación, se presenta el código que calcula el ingreso corriente total promedio por hogar en deciles de hogares y su coeficiente de GINI.
El coeficiente de Gini es una medida de concentración del ingreso: toma valores entre cero y uno. Cuando el valor se acerca a uno, indica que hay mayor concentración del ingreso; en cambio cuando el valor del Gini se acerca a cero la concentración del ingreso es menor.

<blockquote><div style="background-color:#DCDCDC; color:#000000; font-style: normal; font-family: Georgia;">
Folioviv (Folio de la vivienda):

Consta de 10 dígitos: dos dígitos con la clave de la entidad federativa, uno con el ámbito (urbano, código diferente a 6; rural, código 6), cuatro dígitos del número consecutivo de la upm (unidad primaria de muestreo), un dígito de la decena de levantamiento y dos dígitos con un número consecutivo para la vivienda seleccionada.
</div></blockquote>


<blockquote><div style="background-color:#DCDCDC; color:#000000; font-style: normal; font-family: Georgia;">
Foliohog (Folio del hogar):

Dígito para identificar a los hogares en la vivienda: se asigna 1 al hogar principal (el que inicialmente
atendió la entrevista) o cuando en la vivienda solo habita un hogar; cuando se detecten varios hogares
(hogares adicionales) se les asignan dígitos del 2 al 5.
</div></blockquote>

In [5]:
# Seleccionamos variables de interes

concentrado = df[['folioviv', 'foliohog', 'ing_cor', 'ingtrab', 'trabajo',
                  'negocio', 'otros_trab', 'rentas', 'utilidad', 'arrenda',
                 'transfer', 'jubilacion', 'becas', 'donativos', 'remesas',
                 'bene_gob', 'transf_hog', 'trans_inst', 'estim_alqu',
                 'otros_ing', 'factor', 'upm', 'est_dis']]

In [ ]:
concentrado.info()

In [12]:
# Se crea una variable de entidad federativa y de total de integrantes

concentrado['entidad']=df.loc[:,('folioviv')].apply(str).str[1:3]
concentrado['ing_cor']= df['ing_cor']* df['factor']
concentrado['integrantes'] = df['tot_integ']* df['factor']

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [7]:
print('Total de Hogares: ', concentrado.factor.sum())

print('Tamaño de los deciles: ',  concentrado.factor.sum()/10 )

print('Promedio de integrantes por familia: {:.3} '.format(df.tot_integ.mean()))

Total de Hogares:  35749659
Tamaño de los deciles:  3574965.9
Promedio de integrantes por familia: 3.55 


Para crear los deciles de ingreso, utilizamos la función *weighted_qcut* definida al inicio del notebook para obtener los deciles ponderados por el factor de expansión

In [8]:
concentrado['decil']=weighted_qcut(concentrado['ing_cor'], concentrado['factor'], 10, labels=False)

In [13]:
concentrado.groupby(by='decil').agg({'ing_cor': ['sum','min', 'max'],
                                    'integrantes': ['sum', 'mean']})

ing_cor                               integrantes         
                     sum          min              max         sum     mean
decil                                                                      
0      35,524,500,228.01         0.00    56,298,642.20     9623550 1,002.98
1      60,283,958,149.91   170,633.76    63,577,207.56    10547718 1,105.98
2      79,626,976,368.50   237,540.72    83,607,027.13    11324441 1,225.72
3      98,510,524,833.61   249,381.30   149,082,934.52    12064775 1,338.75
4     119,296,646,868.68   316,325.30   179,699,223.12    12660011 1,395.97
5     143,370,324,397.34   368,829.10   200,037,814.68    13292577 1,521.41
6     173,947,987,535.87   448,364.70   183,228,204.55    14049753 1,585.21
7     216,690,070,387.21   557,061.30   323,592,107.16    14347363 1,640.64
8     287,555,799,573.02   760,313.70   319,580,779.84    14590067 1,744.39
9     583,734,001,302.97 1,014,807.80 7,192,058,041.30    14260601 1,817.79

In [11]:
# Exportamos a excel
concentrado.groupby(by='decil').agg({'ing_cor': ['sum','min', 'max'],
                                    'integrantes': ['sum', 'mean']}).to_excel(outputs + 'data/deciles.xlsx')